# Tratamento da tabela de resultados de 2023

base de dados: https://drive.google.com/file/d/1une3K_beXOU-CpPwr49M0w5XQOig0GZe/view?usp=sharing

In [1]:
import pandas as pd

importando tabela de resultados de 2023

rode apenas uma das célular abaixo que definem o dataframe para tratar o arquivo de sua escolha 

In [20]:
df = pd.read_csv("RESULTADOS_02_03_2024_full_teste.csv", compression="gzip")

In [21]:
df = pd.read_csv("RESULTADOS_06_2023_07_2023_full_teste.csv", compression="gzip")

In [32]:
df = pd.read_csv("RESULTADOS_08_2023_09_2023_full_teste.csv", compression="gzip")

In [43]:
df = pd.read_csv("RESULTADOS_10_2023_11_2023_full_teste.csv", compression="gzip")

In [44]:
df.head()

,Unnamed: 0,KNR,NAME,ID,STATUS,UNIT,VALUE_ID,VALUE,DATA
0,0,2023-2316417,012001,1,10,NaN,0.0,NaN,2023-11-07 13:05:08
1,1,2023-2316417,012002,1,10,NaN,0.0,NaN,2023-11-07 13:23:11
2,2,2023-2316417,012003,1,10,NaN,0.0,NaN,2023-11-07 13:23:18
3,3,2023-2316417,012004,1,10,NaN,0.0,NaN,2023-11-07 13:22:00
4,4,2023-2316417,018001,1,10,NaN,0.0,NaN,2023-11-07 14:06:56


Ajustando tabela com colunas apropriadas

In [46]:
df = df.drop(columns=["Unnamed: 0"])

KeyError: "['Unnamed: 0'] not found in axis"

In [47]:
df = df.drop_duplicates()

In [48]:
df.shape

(14633729, 8)

convertendo para datetime para tratar o tempo

In [49]:
df['DATA'] = pd.to_datetime(df['DATA'], errors='coerce')

Pivotando a tabela para criar colunas de acordo com o id e status

In [50]:
pivot_df = df.pivot_table(index='KNR', columns=["ID", "STATUS"], aggfunc='size', fill_value=0)

pivot_df.columns = [f'QTD_STATUS_{col[0]}_OK' if col[1]== 10 else f'QTD_STATUS_{col[0]}_NOK' for col in pivot_df.columns]

pivot_df.reset_index(inplace=True)

Calculando tempo médio

In [51]:
pivot_df["TEMPO_MEDIO"] = (df.groupby('KNR').DATA.transform('max') - df.groupby('KNR').DATA.transform('min')).dt.total_seconds()/60

exportando para parquet

In [52]:
pivot_df.head()

,KNR,QTD_STATUS_1_OK,QTD_STATUS_1_NOK,QTD_STATUS_2_OK,QTD_STATUS_2_NOK,QTD_STATUS_718_OK,QTD_STATUS_718_NOK,TEMPO_MEDIO
0,2023-2316417,10,0,846,15,41,1,18991.283333
1,2023-2316420,12,0,850,4,41,1,18991.283333
2,2023-3276018,0,0,8,0,2,0,18991.283333
3,2023-3276025,0,0,4,0,25,0,18991.283333
4,2023-3476004,1,0,0,0,2,0,18991.283333


In [53]:
pivot_df.to_parquet("RESULTADOS_10_2023_11_2023_TRATADO.parquet")